# Imports

In [1]:
# Library
import requests
import os
from dotenv import load_dotenv

### Import env variables
load_dotenv(dotenv_path="../secrets/app_secrets.env", verbose=True)
INBOUND_CHECK_TOKEN = os.getenv("INBOUND_CHECK_TOKEN")
# Bearer token
bearer_token = "Bearer " + INBOUND_CHECK_TOKEN

# inbound/check
Variables from this cell will be used for the other two endpoints

In [2]:
# mc_core_dev
# host = "http://13.246.32.186:9902"

# mc_core staging
host = "http://13.245.109.60:9902"

In [3]:
#### Test the check endpoint
response = requests.post(
    host + "/inbound/check",
    json={"text_to_match": "test message"},  # "return_scoring": "true"},
    headers={"Authorization": bearer_token},
)

#### store info from response
top_response = response.json()["top_responses"][0][0]
# for feedback endpoint
inbound_id = response.json()["inbound_id"]
feedback_secret_key = response.json()["feedback_secret_key"]
# for pagination endpoint (also inbound_id above)
inbound_secret_key = response.json()["inbound_secret_key"]
initial_next_page_url = response.json()["next_page_url"]

#### print results
print("top response:", top_response)
print("check endpoint response time (s):", response.elapsed.total_seconds())

top response: HIV - Testing
check endpoint response time (s): 0.723977


In [4]:
# response.json()

# inbound/feedback

In [5]:
#### Simple function to test the feedback endpoint
def feedback_endpoint_test(inbound_id, feedback_secret_key, feedback):

    feedback_response = requests.put(
        host + "/inbound/feedback",
        json={
            "inbound_id": inbound_id,
            "feedback_secret_key": feedback_secret_key,
            "feedback": feedback,
        },
        headers={"Authorization": bearer_token},
    )

    print("Feedback call response:", feedback_response.text)
    print("Feedback endpoint response time (s):", response.elapsed.total_seconds())

In [6]:
# correct information
feedback_endpoint_test(
    inbound_id=inbound_id,
    feedback_secret_key=feedback_secret_key,
    feedback={"feedback_type": "positive", "faq_id": 0, "page_number": 0},
)

Feedback call response: Success
Feedback endpoint response time (s): 0.723977


In [7]:
# incorrect feedback format
feedback_endpoint_test(
    inbound_id=inbound_id, feedback_secret_key=feedback_secret_key, feedback="Gold Star"
)

Feedback call response: Malformed Feedback JSON
Feedback endpoint response time (s): 0.723977


In [15]:
# incorrect inbound_id
feedback_endpoint_test(
    inbound_id=0,
    feedback_secret_key=feedback_secret_key,
    feedback={"feedback_type": "positive", "faq_id": 0, "page_number": 0},
)

Feedback call response: No Matches
Feedback endpoint response time (s): 0.746548


In [8]:
# incorrect feedback_secret_key
feedback_endpoint_test(
    inbound_id=inbound_id,
    feedback_secret_key="abcdefg",
    feedback={"feedback_type": "positive", "faq_id": 0, "page_number": 0},
)

Feedback call response: Incorrect Feedback Secret Key
Feedback endpoint response time (s): 0.723977


# Test pagination 
#### inbound/inbound_id/page_number

There are two approaches here:
1. Jumping to a page using the page number
2. Using the next and previous page urls returned by each call to flip through pages

## Jump-to-page

In [9]:
# Simple function to jump to a page and return content
def jump_to_page(page_number, inbound_secret_key):

    # call
    response = requests.get(
        url=host + "/inbound/" + inbound_id + "/" + str(page_number),
        params={
            "inbound_secret_key": inbound_secret_key
        },  #!# params, NOT json. Opposite of other endpoint. #!#
        headers={"Authorization": bearer_token},
    )

    # show results
    if response.json()["top_responses"] == []:
        print("No content.")
    else:
        print("top response:", response.json()["top_responses"][0][0])
        # print("all responses:", response.json()["top_responses"])

    print("check endpoint response time (s):", response.elapsed.total_seconds())

    if "prev_page_url" in response.json():
        print("Previous page url:", "found")  # response.json()['prev_page_url']

    if "next_page_url" in response.json():
        print("Next page:", "found")  # response.json()['next_page_url']

### Page 1 should match initial call response

In [10]:
jump_to_page(1, inbound_secret_key)

top response: HIV - Testing
check endpoint response time (s): 0.833947
Next page: found


### Page 2 should be different

In [11]:
jump_to_page(2, inbound_secret_key)

top response: Support - HIV
check endpoint response time (s): 0.738541
Previous page url: found
Next page: found


### !!!! Go to non-existing page !!!!
Note: Should probably throw an error if a non-existing page is attempted to be accessed? Currently returns empty response but also returns a previous page link (which also can lead to an empty page) but no next page link.

In [12]:
jump_to_page(20, inbound_secret_key)

top response: More Conditions - Bleeding Gums
check endpoint response time (s): 0.797554
Previous page url: found
Next page: found


In [13]:
jump_to_page(19, inbound_secret_key)

top response: Preg - Poo colour
check endpoint response time (s): 0.605877
Previous page url: found
Next page: found


## Flip pages (next and previous)

### Next page from initial call should be the same as page 2

In [14]:
#### Test pagination endpoint - flip pages
response = requests.get(
    url=host + initial_next_page_url,
    headers={"Authorization": bearer_token},
)

# show results
print("top response:", response.json()["top_responses"][0][0])
print("check endpoint response time (s):", response.elapsed.total_seconds())

top response: Support - HIV
check endpoint response time (s): 0.678758


### Flip through all available pages

In [15]:
url = initial_next_page_url
page_number = 2

while True:

    # print current page number
    print("Page", page_number)

    #### call
    response = requests.get(
        url=host + url,
        headers={"Authorization": bearer_token},
    )

    #### print results
    print("top response:", response.json()["top_responses"][0][0])
    print("check endpoint response time (s):", response.elapsed.total_seconds())

    #### return next/prev page link
    # Note: The response will simply not include the next/prev page url field if it
    # doesn't exist, resulting in an error if we try to select it from the JSON response.
    if "next_page_url" in response.json():
        url = response.json()["next_page_url"]
        page_number += 1
    else:
        print("ERROR: No next/prev page.")
        break

Page 2
top response: Support - HIV
check endpoint response time (s): 0.697577
Page 3
top response: HIV - Symptoms
check endpoint response time (s): 0.626844
Page 4
top response: Preg - Maternity leave
check endpoint response time (s): 0.725915
Page 5
top response: Baby - Growth - Birth to 3 months
check endpoint response time (s): 0.710188
Page 6
top response: Privacy - Who can see my personal
check endpoint response time (s): 0.65656
Page 7
top response: Baby - Returning period
check endpoint response time (s): 0.722419
Page 8
top response: Preg - Position at birth
check endpoint response time (s): 0.708076
Page 9
top response: Preg - Fetal Month 9
check endpoint response time (s): 0.819355
Page 10
top response: HIV - Nevirapine syrup
check endpoint response time (s): 0.878447
Page 11
top response: COVID-19 - Breastfeeding
check endpoint response time (s): 0.733404
Page 12
top response: Labour - Stages
check endpoint response time (s): 0.705715
Page 13
top response: Support - Abortion